<a href="https://colab.research.google.com/github/dxiong2001/malibu-ml/blob/main/textrank_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
import nltk.tokenize.texttiling as tt
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('brown')
from nltk.corpus import brown

import re

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [39]:
#text='''Santiago is a Shepherd who has a recurring dream which is supposedly prophetic. Inspired on learning this, he undertakes a journey to Egypt to discover the meaning of life and fulfill his destiny. During the course of his travels, he learns of his true purpose and meets many characters, including an “Alchemist”, that teach him valuable lessons about achieving his dreams. Santiago sets his sights on obtaining a certain kind of “treasure” for which he travels to Egypt. The key message is, “when you want something, all the universe conspires in helping you to achieve it.” Towards the final arc, Santiago gets robbed by bandits who end up revealing that the “treasure” he was looking for is buried in the place where his journey began. The end.'''
text='''In late May 1797 Adams met with his cabinet to discuss the situation and to choose a special commission to France. Adams initially proposed that John Marshall and Elbridge Gerry join Pinckney on the commission, but his cabinet objected to the choice of Gerry because he was not a strong Federalist. Francis Dana was chosen instead of Gerry, but he declined to serve, and Adams, who considered Gerry one of the "two most impartial men in America" (he himself being the other), submitted his name to the United States Senate in Dana's stead without consulting his cabinet.[8] Adams, in introducing the matter to Congress, made a somewhat belligerent speech in which he called for a vigorous defense of the nation's neutrality and expansion of the United States Navy, but stopped short of calling for war against France.[9] Congress approved this choice of commissioners, and Adams instructed them to negotiate similar terms to those that had been granted to Britain in the Jay Treaty.[10] The commissioners were also instructed to refuse loans, but to be flexible in the arrangement of payment terms for financial matters.[11] Marshall left for Europe in mid-July to join Pinckney, with Gerry following a few weeks later.[12] The political divisions in the commission's makeup were reflected in their attitudes toward the negotiations: Marshall and Pinckney, both Federalists, distrusted the French, while Gerry (who was then opposed to political parties) was willing to be flexible and unhurried in dealing with them.[13]

The French Republic, established in 1792 at the height of the French Revolution, was by 1797 governed by a bicameral legislative assembly, with a five-member French Directory acting as the national executive.[b] The Directory was undergoing both internal power struggles and struggles with the Council of Five Hundred, the lower chamber of the legislature. Ministerial changes took place in the first half of 1797, including the selection in July of Charles Maurice de Talleyrand as foreign minister.[15] Talleyrand, who had recently spent a few years in the United States, was openly concerned about the establishment of closer ties between the U.S. and Britain. The Directory, generally not well-disposed to American interests, became notably more hostile to them in September 1797, when an internal coup propelled several anti-Americans into power.[16] These leaders, and Talleyrand, viewed President Adams as hostile to their interests, but did not think that there was significant danger of war.[17] In part based on advice imparted to French diplomats by Jefferson, Talleyrand decided to adopt a measured, slow pace to the negotiations.[18]


The American commission arrived in Paris in early October, and immediately requested a meeting with Talleyrand. After an initial brief meeting (in which Talleyrand only provisionally accepted the commissioners' credentials), a longer meeting was held a week later. Talleyrand sought from the commissioners an explanation for the speech Adams had made in May, which had angered Directory members; his motivation was to determine how favorably the commissioners were disposed to the negotiations. If they responded in an unfavorable manner, the Directory would refuse to accept their credentials.[19] The commissioners first learned of Talleyrand's expected demand on October 14 through an indirect channel. They decided that no explanation would be given for Adams' speech.[20] '''
sentences=sent_tokenize(text)

In [40]:
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
sentence_tokens

[['late',
  'may',
  '1797',
  'adams',
  'met',
  'cabinet',
  'discuss',
  'situation',
  'choose',
  'special',
  'commission',
  'france'],
 ['adams',
  'initially',
  'proposed',
  'john',
  'marshall',
  'elbridge',
  'gerry',
  'join',
  'pinckney',
  'commission',
  'cabinet',
  'objected',
  'choice',
  'gerry',
  'strong',
  'federalist'],
 ['francis',
  'dana',
  'chosen',
  'instead',
  'gerry',
  'declined',
  'serve',
  'adams',
  'considered',
  'gerry',
  'one',
  'two',
  'impartial',
  'men',
  'america',
  'submitted',
  'name',
  'united',
  'states',
  'senate',
  'danas',
  'stead',
  'without',
  'consulting',
  'cabinet'],
 ['8',
  'adams',
  'introducing',
  'matter',
  'congress',
  'made',
  'somewhat',
  'belligerent',
  'speech',
  'called',
  'vigorous',
  'defense',
  'nations',
  'neutrality',
  'expansion',
  'united',
  'states',
  'navy',
  'stopped',
  'short',
  'calling',
  'war',
  'france'],
 ['9',
  'congress',
  'approved',
  'choice',
  'commi

In [41]:
w2v=Word2Vec(sentence_tokens,size=1,min_count=1,iter=1000)
sentence_embeddings=[[w2v[word][0] for word in words] for words in sentence_tokens]
max_len=max([len(tokens) for tokens in sentence_tokens])
sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [42]:
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)

In [43]:
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)

In [44]:
top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])

In [45]:
for sent in sentences:
    if sent in top.keys():
        print(sent)

Ministerial changes took place in the first half of 1797, including the selection in July of Charles Maurice de Talleyrand as foreign minister.
[15] Talleyrand, who had recently spent a few years in the United States, was openly concerned about the establishment of closer ties between the U.S. and Britain.
[17] In part based on advice imparted to French diplomats by Jefferson, Talleyrand decided to adopt a measured, slow pace to the negotiations.
After an initial brief meeting (in which Talleyrand only provisionally accepted the commissioners' credentials), a longer meeting was held a week later.


In [46]:
emma = nltk.corpus.gutenberg.words('austen-emma.txt')

In [47]:
def textrank(text):

  #Preproces text
  sentences=sent_tokenize(text)
  sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
  sentences_clean=[re.sub(r'\[\]','',sentence) for sentence in sentences_clean]
  stop_words = stopwords.words('english')
  sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]

  w2v=Word2Vec(sentence_tokens,size=1,min_count=1,iter=1000)
  sentence_embeddings=[[w2v[word][0] for word in words] for words in sentence_tokens]
  max_len=max([len(tokens) for tokens in sentence_tokens])
  sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]

  similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
  for i,row_embedding in enumerate(sentence_embeddings):
      for j,column_embedding in enumerate(sentence_embeddings):
          similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)


  nx_graph = nx.from_numpy_array(similarity_matrix)
  scores = nx.pagerank(nx_graph)
  #print(scores[0])
  top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
  top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])
  
  #print(sentences_clean)
  summ = []
  i=0
  for sent in sentences:
    if sent in top.keys():
        summ.append(re.sub(r'\[\d*\] ','',sent))
        #print(re.sub(r'\[\d*\] ','',sent))
        i=i+1
  return summ

In [48]:
ranked = textrank(text)
ranked

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


['Ministerial changes took place in the first half of 1797, including the selection in July of Charles Maurice de Talleyrand as foreign minister.',
 'Talleyrand, who had recently spent a few years in the United States, was openly concerned about the establishment of closer ties between the U.S. and Britain.',
 'In part based on advice imparted to French diplomats by Jefferson, Talleyrand decided to adopt a measured, slow pace to the negotiations.',
 "After an initial brief meeting (in which Talleyrand only provisionally accepted the commissioners' credentials), a longer meeting was held a week later."]

In [50]:
texttiler = tt.TextTilingTokenizer()
np.empty
def generate(text):
  tokenized = texttiler.tokenize(text)
  l = len(tokenized)
  l = []
  #summ = np.zeros((l,2))
  for t in range(len(tokenized)):
    l.append(textrank(tokenized[t])[0])
    #textrank(tokenized[t])
    #summ[t][0]=textrank(tokenized[t])[0]
    #summ[t][1]=textrank(tokenized[t])#[1]
  return l
print("output:")
raw_text = generate(text)

processed_text = ""

for i in raw_text:
  processed_text = processed_text + i.replace("\n", "") + " "
print(processed_text)

output:


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In late May 1797 Adams met with his cabinet to discuss the situation and to choose a special commission to France. Ministerial changes took place in the first half of 1797, including the selection in July of Charles Maurice de Talleyrand as foreign minister. The American commission arrived in Paris in early October, and immediately requested a meeting with Talleyrand. 
